In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import json
from PIL import Image

In [3]:
DATA_DIR = "./data/coco"

IMG_HEIGHT = 244
IMG_WIDTH = 244

SEED = 1

In [4]:
# Setup GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)


1 Physical GPUs, 1 Logical GPUs


# Load Data

In [5]:
# Load Image data
image_filenames = {}
image_dir = DATA_DIR + "/val2017/"

for filename in os.listdir(image_dir):
    if filename.endswith(".jpg"):
        image_filenames[int(filename[:-4])] = image_dir + filename

In [6]:
# Load bbox/annotations
with open(DATA_DIR + "/annotations/instances_val2017.json") as f:
    _ = json.load(f)
    categories = {entry["id"]:{
            "supercategory": entry["supercategory"],
            "name": entry["name"]
        } for entry in _["categories"]}
    instances = {entry["id"]:{
            "bbox": tuple(entry["bbox"]),
            "category_id": entry["category_id"],
            "image_id": entry["image_id"]
        } for entry in _["annotations"]}

# Environment Setup

In [7]:
TRIGGER_THRESHOLD = .5

In [8]:
from multiagent.infrastructure.env import ObjectLocalizationEnv, BetterRewardEnv, MovingEdgeEnv, HierarchicalZoomEnv, StretchyZoomEnv

In [9]:
model = tf.keras.models.Sequential(tf.keras.applications.VGG16(weights="imagenet", include_top = True).layers[:-2])

In [10]:
og_env = ObjectLocalizationEnv(model, (224, 224), feature_dim = 4096, trigger_threshold = TRIGGER_THRESHOLD)
moving_edge_env = MovingEdgeEnv(model, (244, 244), feature_dim = 4096, transformation_factor = 0.4, trigger_threshold = TRIGGER_THRESHOLD)
zoomzoom_env = HierarchicalZoomEnv(model, (244,244), feature_dim = 4096, trigger_threshold = TRIGGER_THRESHOLD)
stretchyzoom_env = StretchyZoomEnv(model, (244,244), feature_dim = 4096, trigger_threshold = TRIGGER_THRESHOLD)

# Param Setup

In [11]:
from multiagent.infrastructure.trainer import RL_Trainer
from multiagent.agents.dqn_agent import DQN_Agent
from multiagent.util.dqn_utils import EpochSchedule

In [12]:
agent_params = { #TODO!!!!
    "replay_buffer_size": 500,
    "batch_size": 32,
    "gamma": 0.1,
    "epsilon": EpochSchedule(15, final_p = .1, initial_p = 1),
    "dropout": 0.1
}

In [13]:
params = {
    "learning_freq": 1,
    "seed": SEED,
    "agent_class": DQN_Agent,
    "optimizer": tf.keras.optimizers.Adam(learning_rate = 0.001),
    "loss": tf.keras.losses.Huber(),
    "agent_params": agent_params,
    "env": None, #PlaceHolder
    "max_path_length": 200,
    "model_name": "austensux", #PlaceHolder
    "save_freq": 0 #Manually save after every epoch
}

# Curating category-specific data

In [14]:
CATEGORY_ID = 1
CATEGORY_NAME = categories[CATEGORY_ID]["name"]
NUM_EVAL = 100

In [15]:
from multiagent.util.data import find_all_instances, find_image_ids_with_category, process_image_filename, stream_images

category_image_ids = find_image_ids_with_category(instances, CATEGORY_ID)
print("{} images that include at least one instance of '{}'".format(len(category_image_ids), categories[CATEGORY_ID]["name"]))
category_image_ids = sorted(category_image_ids)

2693 images that include at least one instance of 'person'


# Eval setup

In [16]:
MODEL_PATH = "models/moving_edge/epoch2"

In [17]:
loaded_q_func = tf.keras.models.load_model(MODEL_PATH)

In [18]:
from copy import deepcopy

In [19]:
current_agent_params = deepcopy(agent_params)

params['env'] = stretchyzoom_env
params['model_name'] = "moving_edge"
params['agent_params'] = current_agent_params

In [20]:
trainer = RL_Trainer(params=params)

In [21]:
trainer.agent.q_func = loaded_q_func

# Eval

In [22]:
import os
def save_bboxs(image_id, category_name, bboxs, directory, detection):
    if not os.path.exists(directory):
        os.makedirs(directory)
    with open(directory + str(image_id) + ".txt","w+") as f:
        for bbox in bboxs:
            if detection:
                confidence = 1
                f.write("{} {} {} {} {} {}\n".format(category_name, confidence, *bbox))
            else:
                f.write("{} {} {} {} {}\n".format(category_name, *bbox))
        f.write("\n")
    

In [23]:
GROUND_TRUTH_DIR = "eval/{}/groundtruths/".format(params["model_name"])
DETECTIONS_DIR = "eval/{}/detections/".format(params["model_name"])

In [24]:
# current_image_iter = 0
# category_images = stream_images(category_image_ids, image_filenames, limit=NUM_EVAL)

# for image_id, image_tensor in category_images:
#     current_image_iter += 1
#     print("\n\n####### Testing image '{}' which is {} of {}".format(image_id, current_image_iter, NUM_EVAL))

#     keys, target_bboxs = find_all_instances(instances, image_id, CATEGORY_ID)
#     image_tensor = tf.expand_dims(image_tensor, 0)

#     # have agent look for bboxs
#     trainer.env.training_reset(target_bboxs, image_tensor)
#     detections = trainer.eval(1)
#     print("{} unfound instances out of {}".format(len(trainer.env.target_bboxs), len(trainer.env.orig_target_bboxs)))
    
#     # save bboxs
#     save_bboxs(image_id, CATEGORY_NAME, target_bboxs, GROUND_TRUTH_DIR, False)
#     save_bboxs(image_id, CATEGORY_NAME, detections, DETECTIONS_DIR, True)


In [25]:
# Expert sequence distribution

In [ ]:
dist = []

category_images = stream_images(category_image_ids, image_filenames, limit=NUM_EVAL)

current_image_iter = 0
early_terminated = 0
for image_id, image_tensor in category_images:
    current_image_iter += 1
    print("\n\n####### Image '{}' which is {} of {}".format(image_id, current_image_iter, NUM_EVAL))
    keys, target_bboxs = find_all_instances(instances, image_id, CATEGORY_ID)
    image_tensor = tf.expand_dims(image_tensor, 0)
    
    trainer.env.training_reset(target_bboxs, image_tensor)
    
    done = False
    steps_since_trigger = 0
    for step in range(200):
        steps_since_trigger += 1
        positive_action_idxs = trainer.env._positive_actions_idx()
        if (trainer.env.get_ac_dim() - 1) in positive_action_idxs:
            action = trainer.env.actions[-1]
            dist.append(steps_since_trigger)
            print(steps_since_trigger, end = ", ")
            steps_since_trigger = 0
        elif steps_since_trigger > 40:
            trainer.env.reset()
            early_terminated += 1
            steps_since_trigger = 0
            continue
        else:
            action = trainer.env.get_random_expert_action()
        
        _, _, done = trainer.env.step(action)
        
        if done:
            break
    print("\n{} unfound instances out of {}".format(len(trainer.env.target_bboxs), len(trainer.env.orig_target_bboxs)))



####### Image '139' which is 1 of 100
22, 
1 unfound instances out of 2


####### Image '785' which is 2 of 100
3, 
0 unfound instances out of 1


####### Image '872' which is 3 of 100
3, 4, 
0 unfound instances out of 2


####### Image '885' which is 4 of 100
20, 5, 
6 unfound instances out of 8


####### Image '1000' which is 5 of 100
7, 13, 6, 
9 unfound instances out of 12


####### Image '1268' which is 6 of 100
8, 
3 unfound instances out of 4


####### Image '1296' which is 7 of 100
1, 17, 
0 unfound instances out of 2


####### Image '1353' which is 8 of 100
6, 5, 26, 10, 8, 10, 
0 unfound instances out of 6


####### Image '1490' which is 9 of 100

1 unfound instances out of 1


####### Image '1584' which is 10 of 100
12, 
10 unfound instances out of 11


####### Image '1761' which is 11 of 100
19, 33, 
3 unfound instances out of 5


####### Image '2006' which is 12 of 100
24, 
2 unfound instances out of 3


####### Image '2153' which is 13 of 100
23, 20, 7, 
1 unfound insta

In [ ]:
plt.hist(dist, bins = np.arange(0,41), )
plt.title("Stretchy Zoom ({} instances)".format(len(dist)))
plt.xlabel("Number of steps to localize agent")
plt.ylabel("Frequency")
plt.savefig("stretchy_zoom_dist.png")

In [ ]:
early_terminated